# Using the model server to do an inference using GRPC

## First, make sure we have the libraries we need
Have a look at requirements.txt to see which libraries are used.

In [6]:
!pip install -qr requirements.txt

# Only if you don't have PyTorch installed
#!pip install -q -r requirements-torch-cpu.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp-tekton 1.5.9 requires kfp<1.8.23,>=1.8.10, but you have kfp 2.5.0 which is incompatible.
elyra-server 3.15.0 requires kfp!=1.7.2,<2.0,>=1.7.0, but you have kfp 2.5.0 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install protobuf==3.20.2

## Import the libraries and module where we have all helper code for doing the inference

In [7]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image

from remote_infer_grpc import ort_v5

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/opt/app-root/lib64/python3.9/site-packages/google/protobuf/internal/__init__.py)

## Enter the grpc host and port number, the model name and the YAML file containing your classes

In [ ]:
grpc_host = 'modelmesh-serving.object-detection-yolov5.svc.cluster.local'
grpc_port = 8033
model_name = 'object-detection-yolov5-model'
classes_file = 'coco.yaml'

## Now set the parameters for the inference

In [ ]:
# 1. The image you want to analyze
image_path='images/zidane.jpg'
#image_path='images/bus.jpg' # You can replace this with an image you upload

# 2. Confidence threshold, between 0 and 1 (detections with less score won't be retained)
conf = 0.2

# 3. Intersection over Union Threshold, between 0 and 1 (cleanup overlapping boxes)
iou = 0.6

## Launch the inference and show the result

In [ ]:
infer=ort_v5(grpc_host, grpc_port, model_name, 640, classes_file)
img, out, result = infer(image_path, conf, iou)
print(f'{result}')
print('Predictions:')
print(out)
print('Format: each detection is a float64 array shaped as [top_left_corner_x, top_left_corner_y, bottom_right_corner_x, bottom_right_corner_y, confidence, class_index]')
print('The coordinates are relative to a letterboxed representation of the image of size 640x640')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(img);

## Time it on multiple images!

In [ ]:
import os
import time

folder_path = 'images'
infer=ort_v5(grpc_host, grpc_port, model_name, 640, classes_file)

start_time = time.time()

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Get the full path of the file
    file_path = os.path.join(folder_path, filename)

    # Check if the file path is a file (not a directory)
    if os.path.isfile(file_path):
        # Do the inference on the file
        img, out, result = infer(file_path, conf, iou)
        print(f'{result}')

end_time = time.time()
execution_time = end_time - start_time
print(f"Executed in {execution_time:.2f} seconds")